In [ ]:
library(mfGARCH)
library(xts)
library(readxl)
library(rumidas)
library(lubridate)
library(xts)
library(openxlsx)
library(psych)
library(tseries)
library(gridExtra)
library(ggplot2)

In [2]:
load_low_frequency_data <- function(file_path, sheet_name = NULL, column_name) {

    df <- if (is.null(sheet_name)) {
      read_excel(file_path)
    } else {
      read_excel(file_path, sheet = sheet_name)
    }
    
    if (sheet_name == '' || sheet_name == '') {

        df$EPU <- as.numeric(df[[column_name]])
        df <- df[,c("Date", "EPU")]
    } else {
            
      # Assuming 'year' and 'month' are in separate columns
        df$Year <- as.character(df$Year)
        df$Month <- sprintf("%02d", as.integer(df$Month))
      
      # We create a 'Date_Y_M' assuming the day is always the first of the month
        df$Date_Y_M <- paste(df$Year, df$Month, "01", sep = "-")
        df$Date_Y_M <- as.Date(df$Date_Y_M, format = "%Y-%m-%d")
        df$Date_Y_M <- format(df$Date_Y_M, "%Y-%m") 

        df$EPU <- as.numeric(df[[column_name]])
        # Return the data frame with only the 'Date_Y_M' and 'EPU' columns
        df <- df[, c("Date_Y_M", "EPU")]
    }
    
    
    # Convert 'EPU' to numeric, assuming 'EPU' is already in a correct format that can be converted

    df$EPU_Change_Rate <- c(NA, diff(log(df$EPU)))
    # df$EPU_Change_Rate <- df$EPU
    # df$EPU_Change_Rate <- c(NA, diff(df$EPU) / df$EPU[-length(df$EPU)])
    
  
  return(df)
}


In [3]:
USA_file_path <- "D:/Study/Charlse University/New_Master_Thesis/Final_Model/EPU_Data/US_Policy_Uncertainty_Data.xlsx" # nolint: line_length_linter.
USA_EPU <- load_low_frequency_data(USA_file_path, 'Main Index', 'Three_Component_Index')
USA_EPU <- na.omit(USA_EPU)

Russia_file_path <- "D:/Study/Charlse University/New_Master_Thesis/Final_Model/EPU_Data/Russia_Policy_Uncertainty_Data.xlsx" # nolint: line_length_linter.
Russia_EPU <- load_low_frequency_data(Russia_file_path, 'Russian News-Based Index', 'News-Based Policy Uncertainty Index')
Russia_EPU <- na.omit(Russia_EPU)

Korea_file_path <- "D:/Study/Charlse University/New_Master_Thesis/Final_Model/EPU_Data/Korea_Policy_Uncertainty_Data.xlsx" # nolint: line_length_linter.
Korea_EPU <- load_low_frequency_data(Korea_file_path, 'SK EPU Index Entire Period', 'New South Koean EPU Index')
Korea_EPU <- na.omit(Korea_EPU)

Japan_file_path <- "D:/Study/Charlse University/New_Master_Thesis/Final_Model/EPU_Data/Japan_Policy_Uncertainty_Data.xlsx" # nolint: line_length_linter.
Japan_EPU <- load_low_frequency_data(Japan_file_path, 'Main Index', 'News-based Economic Policy Uncertainty Index')
Japan_EPU <- na.omit(Japan_EPU)

HK_file_path <- "D:/Study/Charlse University/New_Master_Thesis/Final_Model/EPU_Data/HK_EPU_Data_Annotated.xlsx" # nolint: line_length_linter.
HK_EPU <- load_low_frequency_data(HK_file_path, 'HKEPU', 'Monthly_HKEPU_Index')
HK_EPU <- na.omit(HK_EPU)

Germany_file_path <- "D:/Study/Charlse University/New_Master_Thesis/Final_Model/EPU_Data/Europe_Policy_Uncertainty_Data.xlsx" # nolint: line_length_linter.
Germany_EPU <- load_low_frequency_data(Germany_file_path, 'European News-Based Index', 'Germany_News_Index')
Germany_EPU <- na.omit(Germany_EPU)

UK_file_path <- "D:/Study/Charlse University/New_Master_Thesis/Final_Model/EPU_Data/Europe_Policy_Uncertainty_Data.xlsx" # nolint: line_length_linter.
UK_EPU <- load_low_frequency_data(UK_file_path, 'European News-Based Index', 'UK_News_Index')
UK_EPU <- na.omit(UK_EPU)


In [ ]:
load_high_frequency_data <- function(file_path, sheet_name = NULL, column_name) {

    df <- if (is.null(sheet_name)) {
      read_excel(file_path, col_types = "text")
    } else {
      read_excel(file_path, sheet = sheet_name, col_types = "text")
    }
    
    if (sheet_name == '' || sheet_name == '') {

        df$Close <- as.numeric(df[[column_name]])
        df <- df[,c("Date", "Close")]
    } else {
            
      # Convert 'Date_Y_M' to Date format
        df$Date_Y_M_D <- as.Date(df$Date, format = "%Y%m%d")
        df$Date_Y_M <- format(df$Date_Y_M_D, format = "%Y-%m")
        df$Date_Y <- format(df$Date_Y_M_D, "%Y")
        df$Date_M <- format(df$Date_Y_M_D, "%m")
        df$Log_Close <- as.numeric(df[[column_name]])
        # Return the data frame with only the 'Date_Y_M' and 'EPU' columns
        df <- df[,c("Date_Y_M_D", 'Date_Y_M', "Date_Y", "Date_M", "Log_Close")]
        names(df)[1] <- "Date_Y_M_D"
    }
    
    # Convert 'EPU' to numeric, assuming 'EPU' is already in a correct format that can be converted
    df$Log_Close <- c(NA, diff(log(df$Log_Close)))*100
  
  return(df)
  
}


In [5]:
CSI300_Energy_file_path <- "D:/Study/Charlse University/New_Master_Thesis/Final_Model/CSI_300_Data/CSI300_Energy.xlsx" # nolint: line_length_linter.
CSI300_Energy <- load_high_frequency_data(CSI300_Energy_file_path, 'main', 'Close')
CSI300_Energy <- na.omit(CSI300_Energy)

CSI300_Communication_file_path <- "D:/Study/Charlse University/New_Master_Thesis/Final_Model/CSI_300_Data/CSI300_Communication.xlsx" # nolint: line_length_linter.
CSI300_Communication <- load_high_frequency_data(CSI300_Communication_file_path, 'main', 'Close')
CSI300_Communication <- na.omit(CSI300_Communication)

CSI300_Consumer_file_path <- "D:/Study/Charlse University/New_Master_Thesis/Final_Model/CSI_300_Data/CSI300_Consumer.xlsx" # nolint: line_length_linter.
CSI300_Consumer <- load_high_frequency_data(CSI300_Consumer_file_path, 'main', 'Close')
CSI300_Consumer <- na.omit(CSI300_Consumer)

CSI300_Consumer_Discretionary_file_path <- "D:/Study/Charlse University/New_Master_Thesis/Final_Model/CSI_300_Data/CSI300_Consumer_Discretionary.xlsx" # nolint: line_length_linter.
CSI300_Consumer_Discretionary <- load_high_frequency_data(CSI300_Consumer_Discretionary_file_path, 'main', 'Close')
CSI300_Consumer_Discretionary <- na.omit(CSI300_Consumer_Discretionary)

CSI300_financial_file_path <- "D:/Study/Charlse University/New_Master_Thesis/Final_Model/CSI_300_Data/CSI300_financial.xlsx" # nolint: line_length_linter.
CSI300_Financial <- load_high_frequency_data(CSI300_financial_file_path, 'main', 'Close')
CSI300_Financial <- na.omit(CSI300_Financial)

CSI300_Industrial_file_path <- "D:/Study/Charlse University/New_Master_Thesis/Final_Model/CSI_300_Data/CSI300_Industrial.xlsx" # nolint: line_length_linter.
CSI300_Industrial <- load_high_frequency_data(CSI300_Industrial_file_path, 'main', 'Close')
CSI300_Industrial <- na.omit(CSI300_Industrial)

CSI300_Information_file_path <- "D:/Study/Charlse University/New_Master_Thesis/Final_Model/CSI_300_Data/CSI300_Information.xlsx" # nolint: line_length_linter.
CSI300_Information <- load_high_frequency_data(CSI300_Information_file_path, 'main', 'Close')
CSI300_Information <- na.omit(CSI300_Information)

CSI300_Materials_file_path <- "D:/Study/Charlse University/New_Master_Thesis/Final_Model/CSI_300_Data/CSI300_Materials.xlsx" # nolint: line_length_linter.
CSI300_Materials <- load_high_frequency_data(CSI300_Materials_file_path, 'main', 'Close')
CSI300_Materials <- na.omit(CSI300_Materials)

CSI300_medicine_file_path <- "D:/Study/Charlse University/New_Master_Thesis/Final_Model/CSI_300_Data/CSI300_medicine.xlsx" # nolint: line_length_linter.
CSI300_Medicine <- load_high_frequency_data(CSI300_medicine_file_path, 'main', 'Close')
CSI300_Medicine <- na.omit(CSI300_Medicine)

CSI300_public_utility_file_path <- "D:/Study/Charlse University/New_Master_Thesis/Final_Model/CSI_300_Data/CSI300_public_utility.xlsx" # nolint: line_length_linter.
CSI300_Public <- load_high_frequency_data(CSI300_public_utility_file_path, 'main', 'Close')
CSI300_Public <- na.omit(CSI300_Public)

CSI300_Real_Estate_file_path <- "D:/Study/Charlse University/New_Master_Thesis/Final_Model/CSI_300_Data/CSI300_Real_Estate.xlsx" # nolint: line_length_linter.
CSI300_Real_Estate <- load_high_frequency_data(CSI300_Real_Estate_file_path, 'main', 'Close')
CSI300_Real_Estate <- na.omit(CSI300_Real_Estate)



In [ ]:
calculate_rv <- function(data0) {

    data0$date <- as.Date(data0$Date_Y_M_D)

    # Calculate the yield square, RV+, RV-
    data0$RV <- data0$Log_Close^2
    data0$RV_plus <- ifelse(data0$Log_Close > 0, data0$Log_Close^2, 0)
    data0$RV_minus <- ifelse(data0$Log_Close < 0, data0$Log_Close^2, 0)

    # Calculate monthly groupings
    data0$Month <- as.Date(cut(data0$date, breaks = "month"))

        # Initialize the monthly data frame
        unique_months <- unique(data0$Month)
        monthly_data <- data.frame(
        Month = unique_months,
        RV = numeric(length(unique_months)),
        RV_plus = numeric(length(unique_months)),
        RV_minus = numeric(length(unique_months))
    )

    # Calculate the sum of RV, RV_plus, and RV_minus for each month
    for (i in 1:length(unique_months)) {
        month_data <- subset(data0, Month == unique_months[i])
        monthly_data$RV[i] <- sum(month_data$RV, na.rm = TRUE)
        monthly_data$RV_plus[i] <- sum(month_data$RV_plus, na.rm = TRUE)
        monthly_data$RV_minus[i] <- sum(month_data$RV_minus, na.rm = TRUE)
    }

    # Calculate log RV
    monthly_data$LRV <- log(monthly_data$RV)

    # Date Format
    monthly_data$Date_Y_M <- format(monthly_data$Month, format = "%Y-%m")

    return(monthly_data)
}

In [7]:
CSI300_Energy_RV <- calculate_rv(CSI300_Energy)
CSI300_Communication_RV <- calculate_rv(CSI300_Communication)
CSI300_Consumer_RV <- calculate_rv(CSI300_Consumer)
CSI300_Consumer_Discretionary_RV <- calculate_rv(CSI300_Consumer_Discretionary)
CSI300_Financial_RV <- calculate_rv(CSI300_Financial)
CSI300_Industrial_RV <- calculate_rv(CSI300_Industrial)
CSI300_Information_RV <- calculate_rv(CSI300_Information)
CSI300_Materials_RV <- calculate_rv(CSI300_Materials)
CSI300_Medicine_RV <- calculate_rv(CSI300_Medicine)
CSI300_Public_RV <- calculate_rv(CSI300_Public)
CSI300_Real_Estate_RV <- calculate_rv(CSI300_Real_Estate)

In [8]:
merge_dataframe <- function(high_frequency_set, low_frequency_set) {

    merge_df <- merge(high_frequency_set, low_frequency_set, by = "Date_Y_M", all.x = TRUE)
    merge_df <- merge_df[order(merge_df$Date_Y_M_D), ]
    merge_df <- na.omit(merge_df)
    
    # merge_df$Date_Y_M <- as.Date(paste0(merge_df$Date_Y_M, "-01"), format="%Y-%m-%d")
    # merge_df$Date_Y_M_D <- as.Date(merge_df$Date_Y_M_D, format="%Y-%m-%d")
    # merge_df$date <- merge_df$Date_Y_M_D

    return(merge_df)
}

In [9]:
CSI300_Energy_with_RV <- merge_dataframe(CSI300_Energy, CSI300_Energy_RV)
CSI300_Communication_with_RV <- merge_dataframe(CSI300_Communication, CSI300_Communication_RV)
CSI300_Consumer_with_RV <- merge_dataframe(CSI300_Consumer, CSI300_Consumer_RV)
CSI300_Consumer_Discretionary_with_RV <- merge_dataframe(CSI300_Consumer_Discretionary, CSI300_Consumer_Discretionary_RV)
CSI300_Financial_with_RV <- merge_dataframe(CSI300_Financial, CSI300_Financial_RV)
CSI300_Industrial_with_RV <- merge_dataframe(CSI300_Industrial, CSI300_Industrial_RV)
CSI300_Information_with_RV <- merge_dataframe(CSI300_Information, CSI300_Information_RV)
CSI300_Materials_with_RV <- merge_dataframe(CSI300_Materials, CSI300_Materials_RV)
CSI300_Medicine_with_RV <- merge_dataframe(CSI300_Medicine, CSI300_Medicine_RV)
CSI300_Public_with_RV <- merge_dataframe(CSI300_Public, CSI300_Public_RV)
CSI300_Real_Estate_with_RV <- merge_dataframe(CSI300_Real_Estate, CSI300_Real_Estate_RV)


In [10]:
CSI300_Energy_USA <- merge_dataframe(CSI300_Energy_with_RV, USA_EPU)
CSI300_Communication_USA <- merge_dataframe(CSI300_Communication_with_RV, USA_EPU)
CSI300_Consumer_USA <- merge_dataframe(CSI300_Consumer_with_RV, USA_EPU)
CSI300_Consumer_Discretionary_USA <- merge_dataframe(CSI300_Consumer_Discretionary_with_RV, USA_EPU)
CSI300_Financial_USA <- merge_dataframe(CSI300_Financial_with_RV, USA_EPU)
CSI300_Industrial_USA <- merge_dataframe(CSI300_Industrial_with_RV, USA_EPU)
CSI300_Information_USA <- merge_dataframe(CSI300_Information_with_RV, USA_EPU)
CSI300_Materials_USA <- merge_dataframe(CSI300_Materials_with_RV, USA_EPU)
CSI300_Medicine_USA <- merge_dataframe(CSI300_Medicine_with_RV, USA_EPU)
CSI300_Public_USA <- merge_dataframe(CSI300_Public_with_RV, USA_EPU)
CSI300_Real_Estate_USA <- merge_dataframe(CSI300_Real_Estate_with_RV, USA_EPU)

CSI300_Energy_Russia <- merge_dataframe(CSI300_Energy_with_RV, Russia_EPU)
CSI300_Communication_Russia <- merge_dataframe(CSI300_Communication_with_RV, Russia_EPU)
CSI300_Consumer_Russia <- merge_dataframe(CSI300_Consumer_with_RV, Russia_EPU)
CSI300_Consumer_Discretionary_Russia <- merge_dataframe(CSI300_Consumer_Discretionary_with_RV, Russia_EPU)
CSI300_Financial_Russia <- merge_dataframe(CSI300_Financial_with_RV, Russia_EPU)
CSI300_Industrial_Russia <- merge_dataframe(CSI300_Industrial_with_RV, Russia_EPU)
CSI300_Information_Russia <- merge_dataframe(CSI300_Information_with_RV, Russia_EPU)
CSI300_Materials_Russia <- merge_dataframe(CSI300_Materials_with_RV, Russia_EPU)
CSI300_Medicine_Russia <- merge_dataframe(CSI300_Medicine_with_RV, Russia_EPU)
CSI300_Public_Russia <- merge_dataframe(CSI300_Public_with_RV, Russia_EPU)
CSI300_Real_Estate_Russia <- merge_dataframe(CSI300_Real_Estate_with_RV, Russia_EPU)

CSI300_Energy_Korea <- merge_dataframe(CSI300_Energy_with_RV, Korea_EPU)
CSI300_Communication_Korea <- merge_dataframe(CSI300_Communication_with_RV, Korea_EPU)
CSI300_Consumer_Korea <- merge_dataframe(CSI300_Consumer_with_RV, Korea_EPU)
CSI300_Consumer_Discretionary_Korea <- merge_dataframe(CSI300_Consumer_Discretionary_with_RV, Korea_EPU)
CSI300_Financial_Korea <- merge_dataframe(CSI300_Financial_with_RV, Korea_EPU)
CSI300_Industrial_Korea <- merge_dataframe(CSI300_Industrial_with_RV, Korea_EPU)
CSI300_Information_Korea <- merge_dataframe(CSI300_Information_with_RV, Korea_EPU)
CSI300_Materials_Korea <- merge_dataframe(CSI300_Materials_with_RV, Korea_EPU)
CSI300_Medicine_Korea <- merge_dataframe(CSI300_Medicine_with_RV, Korea_EPU)
CSI300_Public_Korea <- merge_dataframe(CSI300_Public_with_RV, Korea_EPU)
CSI300_Real_Estate_Korea <- merge_dataframe(CSI300_Real_Estate_with_RV, Korea_EPU)

CSI300_Energy_Japan <- merge_dataframe(CSI300_Energy_with_RV, Japan_EPU)
CSI300_Communication_Japan <- merge_dataframe(CSI300_Communication_with_RV, Japan_EPU)
CSI300_Consumer_Japan <- merge_dataframe(CSI300_Consumer_with_RV, Japan_EPU)
CSI300_Consumer_Discretionary_Japan <- merge_dataframe(CSI300_Consumer_Discretionary_with_RV, Japan_EPU)
CSI300_Financial_Japan <- merge_dataframe(CSI300_Financial_with_RV, Japan_EPU)
CSI300_Industrial_Japan <- merge_dataframe(CSI300_Industrial_with_RV, Japan_EPU)
CSI300_Information_Japan <- merge_dataframe(CSI300_Information_with_RV, Japan_EPU)
CSI300_Materials_Japan <- merge_dataframe(CSI300_Materials_with_RV, Japan_EPU)
CSI300_Medicine_Japan <- merge_dataframe(CSI300_Medicine_with_RV, Japan_EPU)
CSI300_Public_Japan <- merge_dataframe(CSI300_Public_with_RV, Japan_EPU)
CSI300_Real_Estate_Japan <- merge_dataframe(CSI300_Real_Estate_with_RV, Japan_EPU)

CSI300_Energy_HK <- merge_dataframe(CSI300_Energy_with_RV, HK_EPU)
CSI300_Communication_HK <- merge_dataframe(CSI300_Communication_with_RV, HK_EPU)
CSI300_Consumer_HK <- merge_dataframe(CSI300_Consumer_with_RV, HK_EPU)
CSI300_Consumer_Discretionary_HK <- merge_dataframe(CSI300_Consumer_Discretionary_with_RV, HK_EPU)
CSI300_Financial_HK <- merge_dataframe(CSI300_Financial_with_RV, HK_EPU)
CSI300_Industrial_HK <- merge_dataframe(CSI300_Industrial_with_RV, HK_EPU)
CSI300_Information_HK <- merge_dataframe(CSI300_Information_with_RV, HK_EPU)
CSI300_Materials_HK <- merge_dataframe(CSI300_Materials_with_RV, HK_EPU)
CSI300_Medicine_HK <- merge_dataframe(CSI300_Medicine_with_RV, HK_EPU)
CSI300_Public_HK <- merge_dataframe(CSI300_Public_with_RV, HK_EPU)
CSI300_Real_Estate_HK <- merge_dataframe(CSI300_Real_Estate_with_RV, HK_EPU)

CSI300_Energy_Germany <- merge_dataframe(CSI300_Energy_with_RV, Germany_EPU)
CSI300_Communication_Germany <- merge_dataframe(CSI300_Communication_with_RV, Germany_EPU)
CSI300_Consumer_Germany <- merge_dataframe(CSI300_Consumer_with_RV, Germany_EPU)
CSI300_Consumer_Discretionary_Germany <- merge_dataframe(CSI300_Consumer_Discretionary_with_RV, Germany_EPU)
CSI300_Financial_Germany <- merge_dataframe(CSI300_Financial_with_RV, Germany_EPU)
CSI300_Industrial_Germany <- merge_dataframe(CSI300_Industrial_with_RV, Germany_EPU)
CSI300_Information_Germany <- merge_dataframe(CSI300_Information_with_RV, Germany_EPU)
CSI300_Materials_Germany <- merge_dataframe(CSI300_Materials_with_RV, Germany_EPU)
CSI300_Medicine_Germany <- merge_dataframe(CSI300_Medicine_with_RV, Germany_EPU)
CSI300_Public_Germany <- merge_dataframe(CSI300_Public_with_RV, Germany_EPU)
CSI300_Real_Estate_Germany <- merge_dataframe(CSI300_Real_Estate_with_RV, Germany_EPU)

CSI300_Energy_UK <- merge_dataframe(CSI300_Energy_with_RV, UK_EPU)
CSI300_Communication_UK <- merge_dataframe(CSI300_Communication_with_RV, UK_EPU)
CSI300_Consumer_UK <- merge_dataframe(CSI300_Consumer_with_RV, UK_EPU)
CSI300_Consumer_Discretionary_UK <- merge_dataframe(CSI300_Consumer_Discretionary_with_RV, UK_EPU)
CSI300_Financial_UK <- merge_dataframe(CSI300_Financial_with_RV, UK_EPU)
CSI300_Industrial_UK <- merge_dataframe(CSI300_Industrial_with_RV, UK_EPU)
CSI300_Information_UK <- merge_dataframe(CSI300_Information_with_RV, UK_EPU)
CSI300_Materials_UK <- merge_dataframe(CSI300_Materials_with_RV, UK_EPU)
CSI300_Medicine_UK <- merge_dataframe(CSI300_Medicine_with_RV, UK_EPU)
CSI300_Public_UK <- merge_dataframe(CSI300_Public_with_RV, UK_EPU)
CSI300_Real_Estate_UK <- merge_dataframe(CSI300_Real_Estate_with_RV, UK_EPU)

In [11]:
garch_madis <- function(df_financial, K) {

        # Sys.setenv(TZ = "Shanghai",usetz=TRUE)
        # df_financial$Date_m <- as.Date(df_financial$Date_m)
        # df_financial$Date_Y_M <- as.Date(df_financial$Date_Y_M)
        # df_financial$Date <- as.Date(df_financial$Date)
        # df_financial <- df_financial[df_financial$Date < as.Date("2021-12-31"), ]
        # df_financial <- df_financial[, c('Date', 'Log_Return', 'Date_Y_M', 'EPU', 'EPU_Change_Rate')]
        # names(df_financial)[1] <- "date"
        df_financial$date <- df_financial$Date_Y_M_D
        res <- fit_mfgarch(data = df_financial, y = "Log_Close", x = "EPU_Change_Rate", low.freq = "Month", K = K, gamma=FALSE)

        # res <- fit_mfgarch(data = df_financial, y = "Log_Close", x="EPU_Change_Rate", x.two ="RV", low.freq="Month", low.freq.two="Month", K=K, K.two=K)
        return(res)
}

In [16]:
find_lag <- function(obj) {

    k_arr <- c()
    alpha_p_arr <- c()
    beta_p_arr <- c()
    theta_arr <- c()
    theta_p_arr <- c()
    bic_arr <- c()
    log_lik <- c()
    variance_ratio_arr <- c()

    obj <- subset(obj, Date_Y_M_D >= as.Date("2018-01-01"))
    

    for (x in c(12, 16, 20, 24, 28, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43)) {
        K_value <- x
        fit_res_CSI300 <- garch_madis(obj, K_value)

        k_arr <- c(k_arr, K_value)
        alpha_p_arr <- c(alpha_p_arr, fit_res_CSI300$broom.mgarch[2,4])
        beta_p_arr <- c(beta_p_arr, fit_res_CSI300$broom.mgarch[3,4])
        theta_arr <- c(theta_arr, fit_res_CSI300$broom.mgarch[5,2])
        theta_p_arr <- c(theta_p_arr, fit_res_CSI300$broom.mgarch[5,4])
        bic_arr <- c(bic_arr, fit_res_CSI300$bic)
        log_lik <- c(log_lik, fit_res_CSI300$llh)
        variance_ratio_arr <- c(variance_ratio_arr, fit_res_CSI300$variance.ratio)
    }


    test_df <- data.frame(
        K = k_arr,
        alpha_p = alpha_p_arr,
        beta_p = beta_p_arr,
        theta = theta_arr,
        theta_p = theta_p_arr,
        bic = bic_arr,
        log_lik = log_lik,
        variance_ratio_arr = variance_ratio_arr
    )

    return(test_df)
}

In [17]:
Energy_lags <- find_lag(CSI300_Energy_Korea)
Communication_lags <- find_lag(CSI300_Communication_Korea)
Consumer_lags <- find_lag(CSI300_Consumer_Korea)
CS_lags <- find_lag(CSI300_Consumer_Discretionary_Korea)
Fin_lags <- find_lag(CSI300_Financial_Korea)
Industrial_lags <- find_lag(CSI300_Industrial_Korea)
Infor_lags <- find_lag(CSI300_Information_Korea)
Materials_lags <- find_lag(CSI300_Materials_Korea)
Medicine_lags <- find_lag(CSI300_Medicine_Korea)
Public_lags <- find_lag(CSI300_Public_Korea)
Real_Estate_lags <- find_lag(CSI300_Real_Estate_Korea)

[1] "For ensuring numerical stability of the parameter optimization and inversion of the Hessian, it is best to multiply log returns by 100."
[1] "No frequency specified for calculating the variance ratio - default: low.freq = Month"
[1] "For ensuring numerical stability of the parameter optimization and inversion of the Hessian, it is best to multiply log returns by 100."
[1] "No frequency specified for calculating the variance ratio - default: low.freq = Month"
[1] "For ensuring numerical stability of the parameter optimization and inversion of the Hessian, it is best to multiply log returns by 100."
[1] "No frequency specified for calculating the variance ratio - default: low.freq = Month"
[1] "For ensuring numerical stability of the parameter optimization and inversion of the Hessian, it is best to multiply log returns by 100."
[1] "No frequency specified for calculating the variance ratio - default: low.freq = Month"
[1] "For ensuring numerical stability of the parameter optimizat

In [18]:
print("energy")
Energy_lags
print("communication")
Communication_lags
print("Consumer")
Consumer_lags
print("Consumer_Disc")
CS_lags
print("Financial")
Fin_lags
print("Industrial")
Industrial_lags
print("information")
Infor_lags
print("Materials")
Materials_lags
print("medicine")
Medicine_lags
print("Public")
Public_lags
print("real estate")
Real_Estate_lags

[1] "energy"


K,alpha_p,beta_p,theta,theta_p,bic,log_lik,variance_ratio_arr
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
12,0.0223932417,0,-1.9919343,0.82999275,4958.347,-2457.541,9.337248e-01
16,0.0100410700,0,0.6532111,0.53284239,4683.706,-2320.401,7.731430e-01
20,0.0025679015,0,-5.9857933,0.35613018,4457.391,-2207.448,3.509447e+00
24,0.0067273878,0,4.0767085,0.41103059,4232.622,-2095.275,4.311619e+00
28,0.0009269286,0,5.2936644,0.47535987,3940.685,-1949.528,7.892525e+00
30,0.0003104783,0,7.1857010,0.30958577,3808.523,-1883.559,1.238765e+01
31,0.0001687921,0,8.2690413,0.28822844,3720.076,-1839.406,1.990444e+01
32,0.0001933674,0,8.5249327,0.30793516,3664.238,-1811.553,2.009325e+01
33,0.1255808413,0,2.0980999,0.67289656,3606.190,-1782.599,1.007563e+00


[1] "communication"


K,alpha_p,beta_p,theta,theta_p,bic,log_lik,variance_ratio_arr
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
12,1.049967e-03,0,-1.2713300,0.6946490,5307.755,-2632.245,0.5280214
16,3.760965e-04,0,-0.6807816,0.8215666,4923.498,-2440.297,0.1734800
20,8.056032e-05,0,-3.0734692,0.3627510,4561.909,-2259.707,2.4888248
24,1.744548e-03,0,-3.0767064,0.3932566,4271.331,-2114.629,2.4108811
28,2.188399e-03,0,-4.0243826,0.1920747,3870.503,-1914.437,9.9221678
30,1.156751e-03,0,-4.2706101,0.1715827,3709.240,-1833.918,10.2048505
31,1.343065e-03,0,-5.2410404,0.1240022,3595.369,-1777.052,15.1604605
32,3.931228e-04,0,-5.9055967,0.0848681,3522.056,-1740.462,20.4832736
33,1.645728e-02,0,-4.6940202,0.2009007,3446.786,-1702.897,10.2611649


[1] "Consumer"


...

K,alpha_p,beta_p,theta,theta_p,bic,log_lik,variance_ratio_arr
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
12,1.169288e-05,0,-1.3333546,0.5555248,5169.544,-2563.140,3.887922
16,1.527781e-06,0,-0.6988885,0.3329621,4843.847,-2400.471,2.801165
20,1.856291e-07,0,-2.8589845,0.2224583,4553.576,-2255.540,9.750160
24,1.424253e-05,0,-0.6952736,0.2850628,4287.253,-2122.590,3.210984
28,1.155965e-05,0,-1.6293265,0.1987249,3974.359,-1966.365,4.951845
30,2.937689e-05,0,-0.9654640,0.3537008,3824.828,-1891.712,3.211101
31,1.591491e-05,0,-1.1180491,0.4630531,3711.660,-1835.198,3.625148
32,5.532502e-06,0,-1.3807649,0.2514230,3625.778,-1792.323,5.892460
33,4.887674e-06,0,-0.9963754,0.3180716,3537.546,-1748.277,3.538164


[1] "Public"


K,alpha_p,beta_p,theta,theta_p,bic,log_lik,variance_ratio_arr
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
12,0.07705643,0.000000e+00,0.9087161,0.79602199,4038.320,-1997.527,0.39473657
16,0.05656550,0.000000e+00,2.7389591,0.52834764,3812.182,-1884.639,1.78884708
20,0.06912981,0.000000e+00,0.6648054,0.86959042,3600.409,-1778.957,0.16052187
24,0.20306486,3.794326e-04,5.8913125,0.21213642,3424.312,-1691.120,14.81433724
28,0.15984895,3.437588e-06,7.6707221,0.08758416,3197.328,-1577.849,25.44248983
30,0.30949866,8.905634e-04,8.8932889,0.10124318,3093.874,-1526.235,28.27641107
31,0.59278694,9.543641e-02,8.1413659,0.41301308,3011.876,-1485.306,24.41511508
32,0.68498245,1.853924e-01,7.4797326,0.68362046,2952.904,-1455.886,18.58235714
33,0.55915862,5.356117e-02,7.8938157,0.54403307,2886.177,-1422.593,20.20977860


[1] "real estate"


K,alpha_p,beta_p,theta,theta_p,bic,log_lik,variance_ratio_arr
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
12,3.316804e-01,0.0000000,-0.2236762,0.99538843,5709.330,-2833.033,0.03772785
16,1.664586e-01,0.0000000,6.7984284,0.76625033,5368.283,-2662.690,4.80758625
20,1.323990e-01,0.0000000,1.3418916,0.75597838,5061.210,-2509.358,0.50282988
24,2.072208e-03,0.0000000,4.3791540,0.33649899,4737.009,-2347.468,6.62065030
28,1.372121e-01,0.0000000,27.4239024,0.06066503,4408.676,-2183.523,17.77753568
30,5.938843e-05,0.0000000,5.4751901,0.26998605,4240.672,-2099.634,10.46507214
31,4.660117e-01,0.0000000,6.2041626,0.40629734,4143.773,-2051.254,5.27255850
32,3.839762e-01,0.0000000,6.0413226,0.36274005,4070.826,-2014.847,5.10317047
33,3.378249e-01,0.0000000,6.1412060,0.35854833,3996.139,-1977.574,5.26869805


In [13]:
get_final_result <- function(data_set, k_lag) {
    dd_set <- data_set
    # before <- subset(dd_set, Date_Y_M_D <= as.Date("2017-12-31") & Date_Y_M_D >= as.Date("2013-11-07"))
    # before <- subset(dd_set, Date_Y_M_D >= as.Date("2018-01-01"))
    after <- subset(dd_set, Date_Y_M_D >= as.Date("2018-01-01"))
    # after <- subset(dd_set, Date_Y_M_D <= as.Date("2019-12-31") & Date_Y_M_D >= as.Date("2016-03-01"))

    # before_test <- garch_madis(before, k_lag)
    after_test <- garch_madis(after, k_lag)

    before_test <- ''

    return(list(before_covid = before_test, after_covid = after_test))
}


In [19]:
k_lag <- 36
Select_Country <- "Korea"

CSI300_Energy_Country <- get(paste0("CSI300_Energy_", Select_Country))
CSI300_Communication_Country <- get(paste0("CSI300_Communication_", Select_Country))
CSI300_Consumer_Country <- get(paste0("CSI300_Consumer_", Select_Country))
CSI300_Consumer_Discretionary_Country <- get(paste0("CSI300_Consumer_Discretionary_", Select_Country))
CSI300_Financial_Country <- get(paste0("CSI300_Financial_", Select_Country))
CSI300_Industrial_Country <- get(paste0("CSI300_Industrial_", Select_Country))
CSI300_Information_Country <- get(paste0("CSI300_Information_", Select_Country))
CSI300_Materials_Country <- get(paste0("CSI300_Materials_", Select_Country))
CSI300_Medicine_Country <- get(paste0("CSI300_Medicine_", Select_Country))
CSI300_Public_Country <- get(paste0("CSI300_Public_", Select_Country))
CSI300_Real_Estate_Country <- get(paste0("CSI300_Real_Estate_", Select_Country))

CSI300_Energy_Final_Result <- get_final_result(CSI300_Energy_Country, 37) # germany 37, japan 37, korea 37
CSI300_Energy_Before_Covid <- CSI300_Energy_Final_Result$before_covid
CSI300_Energy_After_Covid <- CSI300_Energy_Final_Result$after_covid

CSI300_Communication_Final_Result <- get_final_result(CSI300_Communication_Country, k_lag)
CSI300_Communication_Before_Covid <- CSI300_Communication_Final_Result$before_covid
CSI300_Communication_After_Covid <- CSI300_Communication_Final_Result$after_covid

CSI300_Consumer_Final_Result <- get_final_result(CSI300_Consumer_Country, k_lag)
CSI300_Consumer_Before_Covid <- CSI300_Consumer_Final_Result$before_covid
CSI300_Consumer_After_Covid <- CSI300_Consumer_Final_Result$after_covid

CSI300_Consumer_Discretionary_Final_Result <- get_final_result(CSI300_Consumer_Discretionary_Country, k_lag)
CSI300_Consumer_Discretionary_Before_Covid <- CSI300_Consumer_Discretionary_Final_Result$before_covid
CSI300_Consumer_Discretionary_After_Covid <- CSI300_Consumer_Discretionary_Final_Result$after_covid

CSI300_Financial_Final_Result <- get_final_result(CSI300_Financial_Country, 37) # russia 35, korea 37
CSI300_Financial_Before_Covid <- CSI300_Financial_Final_Result$before_covid
CSI300_Financial_After_Covid <- CSI300_Financial_Final_Result$after_covid

CSI300_Industrial_Final_Result <- get_final_result(CSI300_Industrial_Country, k_lag)
CSI300_Industrial_Before_Covid <- CSI300_Industrial_Final_Result$before_covid
CSI300_Industrial_After_Covid <- CSI300_Industrial_Final_Result$after_covid

CSI300_Information_Final_Result <- get_final_result(CSI300_Information_Country, k_lag)
CSI300_Information_Before_Covid <- CSI300_Information_Final_Result$before_covid
CSI300_Information_After_Covid <- CSI300_Information_Final_Result$after_covid

CSI300_Materials_Final_Result <- get_final_result(CSI300_Materials_Country, 37)  # USA 在35 上 alpha 会显著, korea 37
CSI300_Materials_Before_Covid <- CSI300_Materials_Final_Result$before_covid
CSI300_Materials_After_Covid <- CSI300_Materials_Final_Result$after_covid

CSI300_Medicine_Final_Result <- get_final_result(CSI300_Medicine_Country, k_lag)
CSI300_Medicine_Before_Covid <- CSI300_Medicine_Final_Result$before_covid
CSI300_Medicine_After_Covid <- CSI300_Medicine_Final_Result$after_covid

CSI300_Public_Final_Result <- get_final_result(CSI300_Public_Country, 38) # Germany 34, russia 38, japan 38, korea 38
CSI300_Public_Before_Covid <- CSI300_Public_Final_Result$before_covid
CSI300_Public_After_Covid <- CSI300_Public_Final_Result$after_covid

CSI300_Real_Estate_Final_Result <- get_final_result(CSI300_Real_Estate_Country, 39) # USA 30, Germany 39 , russia 38, japan 38, korea 39
CSI300_Real_Estate_Before_Covid <- CSI300_Real_Estate_Final_Result$before_covid
CSI300_Real_Estate_After_Covid <- CSI300_Real_Estate_Final_Result$after_covid

current_time <- Sys.time()
print(current_time)

[1] "For ensuring numerical stability of the parameter optimization and inversion of the Hessian, it is best to multiply log returns by 100."
[1] "No frequency specified for calculating the variance ratio - default: low.freq = Month"
[1] "For ensuring numerical stability of the parameter optimization and inversion of the Hessian, it is best to multiply log returns by 100."
[1] "No frequency specified for calculating the variance ratio - default: low.freq = Month"
[1] "For ensuring numerical stability of the parameter optimization and inversion of the Hessian, it is best to multiply log returns by 100."
[1] "No frequency specified for calculating the variance ratio - default: low.freq = Month"
[1] "For ensuring numerical stability of the parameter optimization and inversion of the Hessian, it is best to multiply log returns by 100."
[1] "No frequency specified for calculating the variance ratio - default: low.freq = Month"
[1] "For ensuring numerical stability of the parameter optimizat

In [22]:
print("---------CSI 300 Energy----------")
print(paste( "BIC = ",CSI300_Energy_After_Covid$bic))
print(paste( "variance ratio = ",CSI300_Energy_After_Covid$variance.ratio))
CSI300_Energy_Before_Covid
CSI300_Energy_After_Covid


print("---------CSI 300 Communication -----")
print(paste( "BIC = ",CSI300_Communication_After_Covid$bic))
print(paste( "variance ratio = ",CSI300_Communication_After_Covid$variance.ratio))
CSI300_Communication_Before_Covid
CSI300_Communication_After_Covid


print("---------CSI 300 Consumer -----")
print(paste( "BIC = ",CSI300_Consumer_After_Covid$bic))
print(paste( "variance ratio = ",CSI300_Consumer_After_Covid$variance.ratio))
CSI300_Consumer_Before_Covid
CSI300_Consumer_After_Covid


print("---------CSI 300 Consumer Discretionary -----")
print(paste( "BIC = ",CSI300_Consumer_Discretionary_After_Covid$bic))
print(paste( "variance ratio = ",CSI300_Consumer_Discretionary_After_Covid$variance.ratio))
CSI300_Consumer_Discretionary_Before_Covid
CSI300_Consumer_Discretionary_After_Covid


print("---------CSI 300 Financial -----")
print(paste( "BIC = ",CSI300_Financial_After_Covid$bic))
print(paste( "variance ratio = ",CSI300_Financial_After_Covid$variance.ratio))
CSI300_Financial_Before_Covid
CSI300_Financial_After_Covid


print("---------CSI 300 Industrial -----")
print(paste( "BIC = ",CSI300_Industrial_After_Covid$bic))
print(paste( "variance ratio = ",CSI300_Industrial_After_Covid$variance.ratio))
CSI300_Industrial_Before_Covid
CSI300_Industrial_After_Covid


print("---------CSI 300 Information -----")
print(paste( "BIC = ",CSI300_Information_After_Covid$bic))
print(paste( "variance ratio = ",CSI300_Information_After_Covid$variance.ratio))
CSI300_Information_Before_Covid
CSI300_Information_After_Covid


print("---------CSI 300 Materials -----")
print(paste( "BIC = ",CSI300_Materials_After_Covid$bic))
print(paste( "variance ratio = ",CSI300_Materials_After_Covid$variance.ratio))
CSI300_Materials_Before_Covid
CSI300_Materials_After_Covid


print("---------CSI 300 Medicine -----")
print(paste( "BIC = ",CSI300_Medicine_After_Covid$bic))
print(paste( "variance ratio = ",CSI300_Medicine_After_Covid$variance.ratio))
CSI300_Medicine_Before_Covid
CSI300_Medicine_After_Covid


print("---------CSI 300 Public -----")
print(paste( "BIC = ",CSI300_Public_After_Covid$bic))
print(paste( "variance ratio = ",CSI300_Public_After_Covid$variance.ratio))
CSI300_Public_Before_Covid
CSI300_Public_After_Covid


print("---------CSI 300 Real Estate -----")
print(paste( "BIC = ",CSI300_Real_Estate_After_Covid$bic))
print(paste( "variance ratio = ",CSI300_Real_Estate_After_Covid$variance.ratio))
CSI300_Real_Estate_Before_Covid
CSI300_Real_Estate_After_Covid


current_time <- Sys.time()
print(current_time)

[1] "---------CSI 300 Energy----------"
[1] "BIC =  3315.07299956266"
[1] "variance ratio =  11.7842447391988"


[1] ""

       term   estimate rob.std.err      p.value opg.std.err  opg.p.value
mu       mu 0.05356395  0.05270408 3.094794e-01  0.07098554 4.505038e-01
alpha alpha 0.05872458  0.02124308 5.702593e-03  0.01898054 1.975202e-03
beta   beta 0.90037112  0.05384084 0.000000e+00  0.03428008 0.000000e+00
m         m 1.04354456  0.14701969 1.265876e-12  0.13103890 1.776357e-15
theta theta 8.20574059 11.38119883 4.709151e-01 15.74546547 6.022627e-01
w2       w2 3.66466319  0.95133550 1.170973e-04  6.83095847 5.916278e-01

[1] "---------CSI 300 Communication -----"
[1] "BIC =  3220.99578882041"
[1] "variance ratio =  9.72368843665038"


[1] ""

...

       term   estimate rob.std.err    p.value  opg.std.err  opg.p.value
mu       mu 0.04877195  0.03444320 0.15677226   0.05314604 3.587777e-01
alpha alpha 0.06977340  0.03720370 0.06073188   0.01446510 1.410118e-06
beta   beta 0.91864576  0.04461004 0.00000000   0.01708912 0.000000e+00
m         m 0.69662145  1.30846747 0.59445250   0.30002847 2.024095e-02
theta theta 0.92807612  9.52196106 0.92235561  31.49506914 9.764919e-01
w2       w2 3.35120499  2.56926803 0.19211690 113.41043180 9.764265e-01

[1] "---------CSI 300 Real Estate -----"
[1] "BIC =  3552.11970173394"
[1] "variance ratio =  5.02015975124221"


[1] ""

       term    estimate rob.std.err     p.value  opg.std.err  opg.p.value
mu       mu -0.11672484  0.09206198 0.204835802  0.104000638 2.617148e-01
alpha alpha  0.03352096  0.01871966 0.073344380  0.008185778 4.221245e-05
beta   beta  0.96647903  0.02123267 0.000000000  0.007818425 0.000000e+00
m         m -1.04790002  0.55544429 0.059214433  0.712673198 1.414590e-01
theta theta  6.16735816  5.69370487 0.278724566 21.483712626 7.740577e-01
w2       w2  4.79682789  1.50510437 0.001437369 17.642352221 7.857045e-01

[1] "2024-11-30 13:47:52 CET"


In [18]:
CSI300_Energy_After_Covid$bic
CSI300_Communication_After_Covid$bic
CSI300_Consumer_After_Covid$bic
CSI300_Consumer_Discretionary_After_Covid$bic
CSI300_Financial_After_Covid$bic
CSI300_Industrial_After_Covid$bic
CSI300_Information_After_Covid$bic
CSI300_Materials_After_Covid$bic
CSI300_Medicine_After_Covid$bic
CSI300_Public_After_Covid$bic

[1] 3315.073

[1] 3220.996

[1] 3235.817

[1] 3044.572

[1] 2757.891

[1] 3012.352

[1] 3343.877

[1] 3176.668

[1] 3303.619

[1] 2609.883

In [19]:
CSI300_Energy_After_Covid_Residuals <- na.omit(CSI300_Energy_After_Covid$df.fitted$residuals)
CSI300_Communication_After_Covid_Residuals <- na.omit(CSI300_Communication_After_Covid$df.fitted$residuals)
CSI300_Consumer_After_Covid_Residuals <- na.omit(CSI300_Consumer_After_Covid$df.fitted$residuals)
CSI300_Consumer_Discretionary_After_Covid_Residuals <- na.omit(CSI300_Consumer_Discretionary_After_Covid$df.fitted$residuals)
CSI300_Financial_After_Covid_Residuals <- na.omit(CSI300_Financial_After_Covid$df.fitted$residuals)
CSI300_Industrial_After_Covid_Residuals <- na.omit(CSI300_Industrial_After_Covid$df.fitted$residuals)
CSI300_Information_After_Covid_Residuals <- na.omit(CSI300_Information_After_Covid$df.fitted$residuals)
CSI300_Materials_After_Covid_Residuals <- na.omit(CSI300_Materials_After_Covid$df.fitted$residuals)
CSI300_Medicine_After_Covid_Residuals <- na.omit(CSI300_Medicine_After_Covid$df.fitted$residuals)
CSI300_Public_After_Covid_Residuals <- na.omit(CSI300_Public_After_Covid$df.fitted$residuals)

In [20]:
Box.test(CSI300_Energy_After_Covid_Residuals, lag = 12, type = "Ljung-Box")
Box.test(CSI300_Communication_After_Covid_Residuals, lag = 12, type = "Ljung-Box")
Box.test(CSI300_Consumer_After_Covid_Residuals, lag = 12, type = "Ljung-Box")
Box.test(CSI300_Consumer_Discretionary_After_Covid_Residuals, lag = 12, type = "Ljung-Box")
Box.test(CSI300_Financial_After_Covid_Residuals, lag = 12, type = "Ljung-Box")
Box.test(CSI300_Industrial_After_Covid_Residuals, lag = 12, type = "Ljung-Box")
Box.test(CSI300_Information_After_Covid_Residuals, lag = 12, type = "Ljung-Box")
Box.test(CSI300_Materials_After_Covid_Residuals, lag = 12, type = "Ljung-Box")
Box.test(CSI300_Medicine_After_Covid_Residuals, lag = 12, type = "Ljung-Box")
Box.test(CSI300_Public_After_Covid_Residuals, lag = 12, type = "Ljung-Box")


	Box-Ljung test

data:  CSI300_Energy_After_Covid_Residuals
X-squared = 17.335, df = 12, p-value = 0.1374



	Box-Ljung test

data:  CSI300_Communication_After_Covid_Residuals
X-squared = 9.8437, df = 12, p-value = 0.6297



	Box-Ljung test

data:  CSI300_Consumer_After_Covid_Residuals
X-squared = 11.301, df = 12, p-value = 0.5033



	Box-Ljung test

data:  CSI300_Consumer_Discretionary_After_Covid_Residuals
X-squared = 12.066, df = 12, p-value = 0.4404



	Box-Ljung test

data:  CSI300_Financial_After_Covid_Residuals
X-squared = 15.111, df = 12, p-value = 0.2354



	Box-Ljung test

data:  CSI300_Industrial_After_Covid_Residuals
X-squared = 9.0838, df = 12, p-value = 0.6958



	Box-Ljung test

data:  CSI300_Information_After_Covid_Residuals
X-squared = 6.616, df = 12, p-value = 0.8819



	Box-Ljung test

data:  CSI300_Materials_After_Covid_Residuals
X-squared = 2.4126, df = 12, p-value = 0.9985



	Box-Ljung test

data:  CSI300_Medicine_After_Covid_Residuals
X-squared = 22.533, df = 12, p-value = 0.03196



	Box-Ljung test

data:  CSI300_Public_After_Covid_Residuals
X-squared = 6.4034, df = 12, p-value = 0.8944


In [21]:
library(FinTS)
ArchTest(CSI300_Energy_After_Covid_Residuals, lags = 36)
ArchTest(CSI300_Communication_After_Covid_Residuals, lags = 36)
ArchTest(CSI300_Consumer_After_Covid_Residuals, lags = 36)
ArchTest(CSI300_Consumer_Discretionary_After_Covid_Residuals, lags = 36)
ArchTest(CSI300_Financial_After_Covid_Residuals, lags = 36)
ArchTest(CSI300_Industrial_After_Covid_Residuals, lags = 36)
ArchTest(CSI300_Information_After_Covid_Residuals, lags = 36)
ArchTest(CSI300_Materials_After_Covid_Residuals, lags = 36)
ArchTest(CSI300_Medicine_After_Covid_Residuals, lags = 36)
ArchTest(CSI300_Public_After_Covid_Residuals, lags = 36)

Warning message:
"package 'FinTS' was built under R version 4.2.3"

Attaching package: 'FinTS'


The following object is masked from 'package:rumidas':

    sp500





	ARCH LM-test; Null hypothesis: no ARCH effects

data:  CSI300_Energy_After_Covid_Residuals
Chi-squared = 35.101, df = 36, p-value = 0.5112



	ARCH LM-test; Null hypothesis: no ARCH effects

data:  CSI300_Communication_After_Covid_Residuals
Chi-squared = 43.019, df = 36, p-value = 0.1959



	ARCH LM-test; Null hypothesis: no ARCH effects

data:  CSI300_Consumer_After_Covid_Residuals
Chi-squared = 35.099, df = 36, p-value = 0.5112



	ARCH LM-test; Null hypothesis: no ARCH effects

data:  CSI300_Consumer_Discretionary_After_Covid_Residuals
Chi-squared = 28.811, df = 36, p-value = 0.7971


...


	ARCH LM-test; Null hypothesis: no ARCH effects

data:  CSI300_Industrial_After_Covid_Residuals
Chi-squared = 34.099, df = 36, p-value = 0.5593



	ARCH LM-test; Null hypothesis: no ARCH effects

data:  CSI300_Information_After_Covid_Residuals
Chi-squared = 22.953, df = 36, p-value = 0.955



	ARCH LM-test; Null hypothesis: no ARCH effects

data:  CSI300_Materials_After_Covid_Residuals
Chi-squared = 39.458, df = 36, p-value = 0.318



	ARCH LM-test; Null hypothesis: no ARCH effects

data:  CSI300_Medicine_After_Covid_Residuals
Chi-squared = 35.302, df = 36, p-value = 0.5016



	ARCH LM-test; Null hypothesis: no ARCH effects

data:  CSI300_Public_After_Covid_Residuals
Chi-squared = 33.887, df = 36, p-value = 0.5694
